In [2]:
import pandas as pd
import numpy as np
import os

import plotly.offline as pyo
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly import tools

from scag_colors import gen_color, scenario_colors, veh_fuel_colors

pyo.init_notebook_mode(connected=True)

#### vehicle dropdown
- select scenario
- select vehicle type
- trace for each combination of scenarion and vehicle type


In [3]:
hdv = pd.read_csv(r'\\STORAGE\E3Data\E3 Projects\SCAG Pathways\PlotlyDev\PATHWAYS_csvs\HDVstocks.csv', header = [0,1], index_col=0).fillna(0)
mdv = pd.read_csv(r'\\STORAGE\E3Data\E3 Projects\SCAG Pathways\PlotlyDev\PATHWAYS_csvs\MDVstocks.csv', header = [0,1], index_col=0).fillna(0)
ldv = pd.read_csv(r'\\STORAGE\E3Data\E3 Projects\SCAG Pathways\PlotlyDev\PATHWAYS_csvs\LDVstocks.csv', header = [0,1], index_col=0).fillna(0)
bus = pd.read_csv(r'\\STORAGE\E3Data\E3 Projects\SCAG Pathways\PlotlyDev\PATHWAYS_csvs\BUSstocks.csv', header = [0,1], index_col=0).fillna(0)

scenarios = list(hdv.columns.get_level_values(0).unique())

scenarios.remove('SB 350 Reference')

In [4]:
### makes a lists of traces for each 

def veh_trace_maker(vtype):
    top_traces = [] ## traces for plotting
    scen = [] ## scenario names for filters 
    for sc in scenarios:
        vehs = vtype[sc]
        ordered_hd = list(vehs.loc[2015].sort_values(ascending = False).index)
        vehs = vehs[ordered_hd]
        vehs_c = vehs.cumsum(1)

        ## add columns that will be used as labels
        for label in ordered_hd:
            vehs_c[label + '_text'] = label + '<br>' + vehs[label].apply(lambda x: str(round(x / 10**3, 0))[:-2]) + 'k Vehicles'

        
        scen_sub = []
        scen.append(scen_sub)
        
        traces_vehicles = []
        for g in vehs.columns.values:
            scen_sub.append(sc)
            
            ## set all traces to not visible except Reference
            if sc == 'Reference':
                visible = True
            else:
                visible = False
            
            traces_vehicles.append({'type' : 'scatter',
                               'x' : vehs.index.values,
                               'y' : vehs_c[g],
                               'text' :  vehs_c[g + '_text'],
                               'mode' : 'lines',
                               'opacity' : .7,
                               'line' : {'color' : veh_fuel_colors(g),
                                         'width' : 0},
                               'name' : g,
                               'hoverinfo' : 'text+x',
                               #'fillcolor' : veh_fuel_colors(g),
                               'fill' : 'tonexty',
                               'visible' : visible
                               })

        top_traces.append(traces_vehicles)
    return top_traces, scen

In [5]:
trace_list, scen = veh_trace_maker(hdv)
## comprehensions to unpack lists of traces and scenario filters into a single list for plotly API
scens = [x for item in scen for x in item]
data = [trace for item in trace_list for trace in item]

In [6]:
## creates an updatemenu object to change the figure
updatemenus = list([
    dict(active=0,
         #showactive = True,  ### these two lines are used if buttons are used instead of slider
         #type = 'buttons',
         steps=list([ 
            dict(label = '<b>Reference</b>',
                 method = 'update',
                 args = [{'visible': ['Reference' in x for x in scens]}, ## filters the traces that are visible
                         {'title': 'Reference Scenario',
                          }]),
            dict(label = '<b>High Electrification</b>',
                 method = 'update',
                 args = [{'visible': ['2016 RTP/SCS' in x for x in scens]},
                         {'title': 'High Electrification Scenario',
                          }]),
            dict(label = '<b>Building Electrification</b>',
                 method = 'update',
                 args = [{'visible': ['High Electrification' in x for x in scens]},
                         {'title': 'Building Electrification Scenario',
                          }]),
            dict(label = '<b>High RNG</b>',
                 method = 'update',
                 args = [{'visible': ['High Renewable Gas' in x for x in scens]},
                         {'title': 'High RNG Scenario',
                          }]),
                            ]),
         x = 0,
         y = 0,
         currentvalue = {'visible' : False}),
])

In [8]:
layout = {'title' : 'Reference Scenario',
          'hovermode' : 'closest',
          'width' : 800,
          'height' : 600,
          'xaxis' : {'title' : ''},
          'yaxis' : {'title' : 'Vehicles'},
          'sliders' : updatemenus}

#data = Data(traces_vehicles)

fig = Figure(data = data, layout = layout)

pyo.iplot(fig)


## toggle by scenario
## toggle by truck types

## subplot version (in progress)
- 2 x 2 grid with LDV, MDV, HDV and Bus
- scenario selector should shange each graph

In [9]:
trace2 = go.Scatter(
    x=[20, 30, 40],
    y=[50, 60, 70],
)

In [10]:
layout = {'title' : 'Reference Scenario',
          'hovermode' : 'closest',
          'width' : 800,
          'height' : 600,
          'xaxis' : {'title' : ''},
          'yaxis' : {'title' : 'Vehicles'},
          'sliders' : updatemenus,
          'height' : 600,
          'width' : 1200}

#data = Data(traces_vehicles)

fig = tools.make_subplots(rows = 1, cols = 2)

fig.append_trace(data[1], 1, 1)
fig.append_trace(data[0], 1, 2)

fig['layout'].update(layout)

#fig = Figure(data = data, layout = layout)

pyo.iplot(fig)


## toggle by scenario
## toggle by truck types

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]

